In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys, os, pickle, utils, math

from tqdm.notebook import tqdm
from datetime import timedelta
#from utils import baseline_SCr

if os.getcwd()[-4:] == "code":
    os.chdir('../')

icu = './data/mimic-iv-2.2-parquet/icu/'
hosp = './data/mimic-iv-2.2-parquet/hosp/'

pd.set_option('mode.chained_assignment',  None) # 경고 off

In [2]:
df = pd.read_parquet('./data/resample/resample_merge.parquet')
len(df.stay_id.unique())

73181

In [3]:
df.sort_values(by=['subject_id','hadm_id','stay_id','charttime'],inplace=True)
df['step'] = df.groupby('stay_id').cumcount()

In [4]:
# 24시간 이내 퇴원
df = df.groupby('stay_id').filter(lambda x:len(x) > 24)
len(df.stay_id.unique())

57734

In [5]:
# 24시간 이내 AKI 발생

# 각 stay_id에 대해 처음 24개 행을 선택
first_24 = df.groupby('stay_id').head(24)[['step','subject_id','hadm_id','stay_id','charttime','uo','AKI_UO','SCr','baseline_SCr','AKI_SCr','AKI']]

# 'AKI' 지표가 1 이상인 행을 포함하는 stay_id를 제외
filtered_df = first_24.groupby('stay_id').filter(lambda x: not (x['AKI'] >= 1).any())

# 원래 DataFrame에서 필터링된 stay_id만 선택
df = df[df['stay_id'].isin(filtered_df['stay_id'].unique())]

len(df.stay_id.unique())

15759

In [6]:
# 각 stay_id별로 count_category가 3 이상인 경우의 개수 계산
count_3_or_more = df.groupby('stay_id')['count_category'].transform(lambda x: (x >= 3).sum())

# count_category가 3 이상인 경우가 없는 stay_id만 필터링
df = df[count_3_or_more < 1]
len(df.stay_id.unique())

15540

In [8]:
df.dropna(subset=['HR','SBP','DBP','MAP','temp','RR','weight'],inplace=True)
len(df.stay_id.unique())

15461

In [9]:
tmp = df.isna().sum()

In [11]:
# Action
df.loc[df['count_category']==0, 'action'] = 0
df.loc[(df['count_category']==1)&(df['Cephalosporins']==1), 'action'] = 1
df.loc[(df['count_category']==1)&(df['Vancomycin']==1), 'action'] = 2
df.loc[(df['count_category']==1)&(df['Betalactam_comb']==1), 'action'] = 3
df.loc[(df['count_category']==1)&(df['Metronidazole']==1), 'action'] = 4
df.loc[(df['count_category']==1)&(df['Carbapenems']==1), 'action'] = 5
df.loc[(df['count_category']==1)&(df['Penicillins']==1), 'action'] = 6
df.loc[(df['count_category']==1)&(df['Fluoroquinolones']==1), 'action'] = 7
df.loc[(df['count_category']==1)&(df['Others']==1), 'action'] = 8

df.loc[(df['count_category']==2), 'action'] = 11
df.loc[(df['count_category']==2)&(df['Cephalosporins']==1)&(df['Vancomycin']==1), 'action'] = 9
df.loc[(df['count_category']==2)&(df['Betalactam_comb']==1)&(df['Vancomycin']==1), 'action'] = 10

In [13]:
df.action.value_counts()

action
0.0     1265435
1.0       30494
2.0       26675
3.0       13468
4.0        8095
6.0        7367
5.0        7308
8.0        6226
7.0        4714
11.0       3779
9.0        2068
10.0        905
Name: count, dtype: int64

In [14]:
df[df['SCr'].isna()].AKI.value_counts()

AKI
0    1358
2      68
1      66
3      28
Name: count, dtype: int64

In [16]:
len(df[df['SCr'].isna()].stay_id.unique())

41